In [9]:
import copy, pandas, pickle

import gumpy, joblib, numpy

from skops.io import load

from misc import construct_line, DataFrameSelector

pandas.options.display.max_columns = 999

Load the TB reference and build a pncA gene

In [2]:
ref = gumpy.Genome('/Users/fowler/packages/tuberculosis_amr_catalogues/catalogues/NC_000962.3/NC_000962.3.gbk')
pnca = ref.build_gene('pncA')

Iterate through the nucleotide sequence of the gene to find all the single SNP missense mutations

In [3]:
bases = ['a', 't', 'c', 'g']

mutations = []

for idx,nuc in zip(pnca.nucleotide_index, pnca.nucleotide_sequence):

    for j in bases:

        if nuc==j:
            continue

        # make the single SNP to a copy
        new_pnca = copy.deepcopy(pnca)
        new_pnca.nucleotide_sequence[new_pnca.nucleotide_index==idx]=j

        # translate 
        new_pnca._translate_sequence()

        # find out the resulting mutation
        gene_diff = pnca - new_pnca
        mut = gene_diff.mutations[0]

        # ignore promoter mutations
        if '-' in mut:
            continue

        # ignore synoymouse mutations
        if mut[0]==mut[-1]:
            continue
        
        # ignore nonsense mutations
        if mut[0]=='!':
            continue
        if mut[-1]=='!':
            continue

        # skip these as they are not resolved in the protein structure
        if mut[1:-1]=='186':
            continue

        # if we haven't seen this mutation before, remember it
        if mut not in mutations:
            mutations.append(mut)

In [4]:
print("There are %i missense mutations." % len(mutations))

There are 1099 missense mutations.


In [5]:
df = pandas.DataFrame(mutations, columns=['MUTATION'])

# we don't know the phenotypes so arbitratily set them all to susceptible
df[['CONSISTENT_PHENOTYPE']] = 'S'

# save to disc
df.to_csv('data/ds-all-phen.csv', index=False)
df[:5]

,MUTATION,CONSISTENT_PHENOTYPE
0,M1L,S
1,M1V,S
2,M1K,S
3,M1T,S
4,M1R,S


At this point return to `06-add-features`, set `filestem = 'all'` and run all cells to add all features to this dataset.

Now load the best XGBoost model

In [6]:
best_model={}
for model in ['LR', 'NN', 'XB']:
    best_model[model] = load('models/'+model.lower()+'.skops', trusted=True)

Load the dataset with features added

In [12]:
df = pandas.read_csv('data/ds-all-phen-features.csv')
df[:3]

,mutation,segid,phenotype,d_volume,d_hydropathy_KD,d_Pi,d_MW,d_rogov,phi,psi,residue_sasa,n_hbond_acceptors,n_hbond_donors,secondary_structure_codes,temp_factor,depth,deep_ddG,rasp_score_ml,snap2_score,dist_FE2,dist_PZA,mcsm_stability_ddG,mapp_score
0,M1I,A,S,3.8,2.6,0.28,-18.0,-0.452,0.0,-35.2,86.2,2.0,0.0,4,59.720001,2.103049,-0.407,0.076292,7,29.948318,27.641663,-0.771,21.99
1,M1K,A,S,5.7,-5.8,4.00,-3.0,-0.712,0.0,-35.2,86.2,2.0,0.0,4,59.720001,2.103049,-1.017,0.621006,70,29.948318,27.641663,-0.214,27.72
2,M1L,A,S,3.8,1.9,0.24,-18.0,-0.389,0.0,-35.2,86.2,2.0,0.0,4,59.720001,2.103049,-0.770,-0.107405,20,29.948318,27.641663,-0.771,13.49


Load the pipeline since we will need to transform the features

In [8]:
pipe = joblib.load('data/pipeline.pkl')

In [13]:
mutations ={}
df = pandas.read_csv('data/ds-all-phen-features.csv')
df.drop(columns=['secondary_structure_codes',
    'd_MW',
    'phi',
    'd_volume',
    'd_Pi',
    'n_hbond_acceptors'],inplace=True)
# make one of the mutations resistant to force it to calculate the statistics
df.loc[(df.mutation=='M1L'), 'phenotype']='R'

mutations['v'] = df['mutation']
features = df.iloc[:,3:]
labels = df['phenotype'].map({'S':0, 'R':1}) #, 'U':2})

X = pipe.transform(features)
Y = labels.squeeze().to_numpy()
Z = mutations['v'].to_numpy()

with open('data/ds-all.npy', 'wb') as f:
    numpy.save(f, Y)
    numpy.save(f, X)
    numpy.save(f, Z)

In [14]:
X={}
Y={}
Z={}

for i in ['all']:
    X[i]={}
    Y[i]={}
    Z[i]={}
    with open('data/ds-'+i+'.npy', 'rb') as f:
        Y[i]['input'] = numpy.load(f)
        X[i]['input'] = numpy.load(f)
        Z[i]['input'] = numpy.load(f, allow_pickle=True)

In [15]:
def validate_model(line, best_model, model_name, X, Y):

    for dataset in ['all']: 
        
        Y[dataset]['predicted'] = best_model[model_name].predict(X[dataset]['input'])
        Y[dataset]['scores'] = best_model[model_name].predict_proba(X[dataset]['input'])[:,1]

        row = construct_line(model_name , dataset, None, Y[dataset], None)
        line.append(row)

    return(line)

line = []
for model in ['LR', 'NN', 'XB']:
    line = validate_model(line, best_model, model, X, Y)

results = pandas.DataFrame(line, columns=['model', 'dataset', 'sensitivity_mean', 'sensitivity_std', 'specificity_mean', 'specificity_std' ,'roc_auc_mean', 'roc_auc_std','TN','FP','FN','TP', 'model_parameters'])
results  

,model,dataset,sensitivity_mean,sensitivity_std,specificity_mean,specificity_std,roc_auc_mean,roc_auc_std,TN,FP,FN,TP,model_parameters
0,LR,all,0.0,None,52.823315,None,31.785064,None,580,518,1,0,null
1,NN,all,100.0,None,47.905282,None,62.750455,None,526,572,0,1,null
2,XB,all,0.0,None,53.278689,None,52.641166,None,585,513,1,0,null
